In [3]:
import os
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.isotonic import IsotonicRegression
import numpy as np
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
from random import randint

plt.rcParams["figure.figsize"] = (20, 16)
warnings.simplefilter("ignore")
files = [i for i in os.listdir("Mesures") if i.endswith(".csv") ]
files

### <8 mbits en cumsum = bruit 

['messenger20221117_125310.csv',
 'messenger20221124_143112.csv',
 'messenger20221124_151137.csv',
 'messenger20221125_130559.csv',
 'messenger20221125_150706.csv',
 'messenger20221129_131204.csv',
 'messenger20221129_151124.csv',
 'skype20221129_164105.csv',
 'twitch20221117_113554.csv',
 'twitch20221118_075454.csv',
 'twitch20221124_163054.csv',
 'twitch20221125_133859.csv',
 'youtube20221108_165214.csv',
 'youtube20221108_182416.csv',
 'youtube20221118_104205.csv',
 'youtube20221124_125524.csv',
 'youtube20221124_154037.csv',
 'youtube20221130_103723.csv']

In [4]:
current_file = 'youtube20221124_154037.csv'
file_path = os.path.join("Mesures", current_file)
data = pd.read_csv(file_path)
#data = pd.read_csv('output.csv')

data

,Time,rnti,SFN,format,preamble,ta,nb_TB,MCSi_1,MCSi_2,TBS_1,TBS_2,nb_PRB,connection_id,label,id_acquisition
0,2022-11-24 14:40:51.871,396,7839,2.0,26.0,4.0,1,1,-2,208,-2,6,4,noise,20221124_154037
1,2022-11-24 14:40:51.896,396,7864,0.0,26.0,4.0,1,12,-2,176,-2,1,4,noise,20221124_154037
2,2022-11-24 14:40:51.904,396,7872,0.0,26.0,4.0,1,12,-2,176,-2,1,4,noise,20221124_154037
3,2022-11-24 14:40:51.912,396,7880,2.0,26.0,4.0,1,17,-2,280,-2,1,4,noise,20221124_154037
4,2022-11-24 14:40:51.914,396,7882,2.0,26.0,4.0,1,1,-2,224,-2,7,4,noise,20221124_154037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62261,2022-11-24 15:23:09.516,396,5964,0.0,26.0,4.0,1,6,-2,256,-2,3,4,noise,20221124_154037
62262,2022-11-24 15:23:09.524,396,5972,2.0,26.0,4.0,1,14,-2,224,-2,1,4,noise,20221124_154037
62263,2022-11-24 15:23:09.551,396,5999,2.0,26.0,4.0,1,29,-2,224,-2,1,4,noise,20221124_154037
62264,2022-11-24 15:23:09.576,396,6024,0.0,26.0,4.0,1,22,-2,1384,-2,3,4,noise,20221124_154037


In [12]:
for filename in tqdm(files) : 
    file_path = os.path.join("Mesures", filename)
    new_data = pd.read_csv(file_path)
    print(file_path , new_data.shape)
  

  6%|████▌                                                                              | 1/18 [00:01<00:21,  1.25s/it]

Mesures\messenger20221117_125310.csv (373020, 15)


 11%|█████████▏                                                                         | 2/18 [00:04<00:35,  2.23s/it]

Mesures\messenger20221124_143112.csv (829766, 15)


 17%|█████████████▊                                                                     | 3/18 [00:08<00:44,  2.99s/it]

...

Mesures\youtube20221118_104205.csv (116938, 15)


 94%|█████████████████████████████████████████████████████████████████████████████▍    | 17/18 [01:03<00:01,  1.24s/it]

Mesures\youtube20221124_125524.csv (100205, 15)
Mesures\youtube20221124_154037.csv (62266, 15)


100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [01:03<00:00,  3.55s/it]

Mesures\youtube20221130_103723.csv (79719, 15)


In [16]:
global_data = pd.concat([pd.read_csv(os.path.join("Mesures",filename)) for filename in tqdm(files) ]) 


100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [01:00<00:00,  3.33s/it]


In [17]:
global_data["uniq_k"] = global_data["id_acquisition"] + global_data["connection_id"].apply(lambda x : "_"+str(x))

global_data.drop(inplace = True, columns = ["SFN", "rnti", "preamble", "MCSi_1", "MCSi_2",
                                            "nb_PRB", "connection_id", "id_acquisition"])
global_data.reset_index(drop = True, inplace = True)
global_data

,Time,format,ta,nb_TB,TBS_1,TBS_2,label,uniq_k
0,2022-11-17 11:53:53.714,0.0,3.0,1,376,-2,noise,20221117_125310_4
1,2022-11-17 11:53:53.722,2.0,3.0,1,224,-2,noise,20221117_125310_4
2,2022-11-17 11:53:53.724,2.0,3.0,1,224,-2,noise,20221117_125310_4
3,2022-11-17 11:53:53.734,0.0,3.0,1,376,-2,noise,20221117_125310_4
4,2022-11-17 11:53:53.762,2.0,3.0,1,224,-2,noise,20221117_125310_4
...,...,...,...,...,...,...,...,...
21428077,2022-11-30 10:21:59.229,0.0,4.0,1,520,-2,noise,20221130_103723_49
21428078,2022-11-30 10:21:59.237,2.0,4.0,1,224,-2,noise,20221130_103723_49
21428079,2022-11-30 10:21:59.252,2.0,4.0,1,224,-2,noise,20221130_103723_49
21428080,2022-11-30 10:21:59.253,2.0,4.0,1,224,-2,noise,20221130_103723_49


In [21]:
global_data.uniq_k.value_counts()

20221129_131204_6     1022014
20221125_150706_31     965580
20221129_131204_10     951170
20221125_150706_32     916104
20221125_150706_34     889853
                       ...   
20221125_150706_60          7
20221125_130559_42          6
20221125_150706_36          4
20221125_150706_37          2
20221117_113554_1           1
Name: uniq_k, Length: 1200, dtype: int64

In [23]:
global_data[["uniq_k", "label"]].value_counts()

uniq_k              label    
20221129_131204_6   messenger    1020766
20221125_150706_31  messenger     961511
20221129_131204_10  messenger     951170
20221125_150706_32  messenger     915076
20221125_150706_34  messenger     889082
                                  ...   
20221124_143112_4   messenger          2
20221124_125524_41  noise              2
20221124_125524_64  noise              2
20221124_125524_20  noise              2
20221117_113554_1   twitch             1
Length: 1303, dtype: int64

In [ ]:
# to apply for each time serie



# format data : 
# 2 colonne = 1time series ( valeurs de TBS + son cumsum ), name col = "{label}_{val or sum}_id"
# les lignes = time codes ( on commence à 0 pour toutes)

def pass_trhough_first_layer(data) :
    list_rows = []
    for i in tqdm(range(len(data.columns))) : # Pour chaque time serie
        useful_columns = data.columns[i]
        new_row = first_layer(data[useful_columns])
        
        new_row["label"] = useful_columns.split("_")[0]
        
        list_rows.append(new_row)
    data_final = pd.DataFrame.from_records(list_rows)
        
    
    # concatenate new rows  
    # return concatenation
    
    
    return data_final
    

def first_layer(data):
    metrics = {}
    cum_data = data.cumsum()
    
    
    
    
    # apply scaler
    
    # Here we apply models on cumulated sum
    models = ["reg_lin", "reg_iso", "reg_pol"]
    
    for model in models : 
        
        metrics = apply_simple_model(model, cum_data, metrics)
        
    # Here we can measure other metrics on none cumulated sum
    
    # Q1 to obtain plateau. median, mean Q3 , other percentiles(10, 90)... same usage I hope.
    # intuition = un plateau = TBS à 0 pendant un certain temps, ainsi x% du temps le TBS vaut 0 donc faible percentile à 0 
    percentiles = [0.1,0.25,0.5,0.75,0.9]
    to_get = {"min" : "p0",
              "10%" : "p10",
              "25%" : "p25",
              "50%" : "p50",
              "75%" : "p75",
              "90%" : "p90",
              "max" : "p100",
              
              "mean" : "mean",
              "std" : "std"}
    
    stats = data.describe(percentiles=percentiles)

    for desc_name, new_name in to_get.items() :
        metrics[new_name] =  stats.loc[desc_name]
    
    return metrics
    
         
    
def apply_simple_model(model, data, metrics) :
    current_data = data.dropna()
    #print(current_data.index.values.reshape(-1, 1))
    X = current_data.index.values.reshape(-1, 1)
    y = current_data.values
    
    if model =="reg_lin" :
        
        reg = LinearRegression()
        reg.fit(X, y)
        
        metrics["reg_lin_R2"] = reg.score(X, y)
        metrics["reg_lin_coef_A"] = reg.coef_[0]
        
    
    elif model =="reg_iso" :
        
        
        pass
    
    elif model == "reg_pol" :
        
        iso_reg = IsotonicRegression(increasing = True)
        iso_reg.fit(X, y)
        
        metrics["reg_iso_R2"] = iso_reg.score(X, y) # non fonctionnel
        
        if False :
            print("\n")
            print("params iso reg ", iso_reg.get_params(deep=True))
            print(iso_reg.X_thresholds_)
            print(iso_reg.y_thresholds_)
            print(len(iso_reg.X_thresholds_))
            print(len(iso_reg.y_thresholds_)," / ", len(y))
    
    
    return metrics

def scale_data(metrics_data):
    x_train_unscaled = metrics_data[list(metrics_data.columns[:-1])].values
    y_train = metrics_data["label"]
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train_unscaled)
    
    new_df = pd.DataFrame(x_train,columns = list(metrics_data.columns[:-1]))
    new_df["label"] = metrics_data['label']
    
    return new_df
    
    
def apply_classifier(metrics_data,model = "KNN", test = True) :
    
    x_train = metrics_data[list(metrics_data.columns[:-1])].values
    y_train = metrics_data["label"]
    
    if test : 
        x_train, x_test, y_train, y_test = train_test_split(x_train, y_test, test_size=0.2,random_state=100)
    
    cv = min(10, int(x_train.shape[0]/3))
    verb = 1
    
    if model == "KNN" :
        
        
        knn = KNeighborsClassifier()
        k_range = list(range(1, 31))
        param_grid = dict(n_neighbors=k_range)
        
        grid = GridSearchCV(knn, param_grid, cv=cv, scoring='accuracy', return_train_score=False,verbose=verb)

        grid_search=grid.fit(x_train, y_train)
        accuracy = grid_search.best_score_ *100
        print("Accuracy for our training dataset with tuning is : {:.2f}%".format(accuracy) )

    
    elif model == "RF":
        
        clf = RandomForestClassifier()
        
        param_grid = {'n_estimators': [15,30, 50, 100],
                      'max_features': ['auto', 'sqrt', 'log2'],
                      'max_depth' : [2,3,4,5,6,7,8],
                      'criterion' :['gini', 'entropy']
                     }
        param_grid = {'n_estimators': [15, 50, 100],
                      'max_features': ['auto', 'sqrt', 'log2'],
                      'criterion' :['gini', 'entropy']
                     }
        
        
        CV_rfc = GridSearchCV(estimator=clf, param_grid=param_grid, cv=cv, scoring ='accuracy', return_train_score=False,verbose=verb)
        CV_rfc.fit(x_train, y_train)
        
        accuracy = CV_rfc.best_score_ *100
        print("Accuracy for our training dataset with tuning is : {:.2f}%".format(accuracy) )
        
        
    if test :
        pass
        
    
    
    